# Obtain edges from a Pajek-like file

In [31]:
import time
import pandas as pd
import numpy as np
from nodes_to_pajek import *
pd.set_option('display.max_rows', 500)

In [2]:
s=time.time()
imax=100#000
for i in range(imax):
    for j in range(imax):
        a=1+1

print(time.time()-s)        

0.001239776611328125


##  Obtain full nodes file.  
Small example and Real case scenario

In [ ]:
FULL=True
if FULL==True:
    df_all=pd.read_json('hep_records.json',lines=True)
else:
    df_all=pd.read_json('nodes_small.json')
    


In [ ]:
dfr=df_all[df_all.references.map(len)>0].reset_index(drop=True)
dfr.shape

In [ ]:
pjt=nodes_to_pajek(dfr)

In [ ]:
pjt.shape

In [ ]:
pj=pjt.reset_index(drop=True)

In [10]:
pj.shape

(21733815, 2)

## Reload and start here

In [463]:
def get_edges(df,i,j):
    dfi=df[np.logical_or(df.recid==i,df.recid==j)]
    dm=dfi.references.values[:-1]-dfi.references.values[1:]
    dfi['nm']=np.concatenate((dm,[1]))
    dfi['np']=np.concatenate(([1],dm))
    return dfi[np.logical_or(dfi.nm==0,dfi.np==0)]

In [474]:
get_edges(pj_full,1403659,1244602)

,recid,references,nm,np


In [464]:
#pj.to_json('pj_full.json')
pj=pd.read_json('pj_full.json')

In [465]:
pj_full=pj.copy()

In [466]:
pj=pj_full

In [467]:
pj.shape

(21733815, 2)

Loop upon references

In [568]:
i,j=625, 307561
pj=pj_full[np.logical_or(pj_full.recid==i,pj_full.recid==j)]

In [569]:
pj.shape

(775, 2)

In [570]:
import warnings
warnings.filterwarnings('ignore')

In [571]:
nmim=2
final_edges=pd.DataFrame()

In [572]:
def get_edges(df_r,i,j):
    dfi=df_r[np.logical_or(df_r.recid==i,df_r.recid==j)]
    dm=dfi.references.values[:-1]-dfi.references.values[1:]
    dfi['nm']=np.concatenate((dm,[1]))
    dfi['np']=np.concatenate(([1],dm))
    return dfi[np.logical_or(dfi.nm==0,dfi.np==0)]

In [573]:
get_edges(pj,i,j).shape

(36, 4)

In [574]:
#New pj
n_i=0
while n_i<100:#= pj.shape[0]>0 or   pj_full.shape[0]:
    print('n_i',n_i)
    n_i=n_i+1
    pj=pj.sort_values(['references','recid']).reset_index(drop=True) 
    if True:
        I=[]
        for r in pj.references:
            pjr=pj[pj.references==r]
            if pjr.shape[0]>1:
                break
            else:
                I=I+list( pjr.index )

        #pjr with at least two references
        #Analize pjr
        for i in pjr.recid.values:
            for j in pjr.recid.values:
                if i<j:
                    print(i,j)
                    #pji=pj[np.logical_or(pj.recid==i,pj.recid==j)]

                    #dm=pji.references.values[:-1]-pji.references.values[1:]
                    #pji['nm']=np.concatenate((dm,[1]))
                    #pji['np']=np.concatenate(([1],dm))
                    #pji=pji[np.logical_or(pji.nm==0,pji.np==0)]
                    pji=get_edges(pj,i,j)

                    II=list( pji.index)
                    II=np.unique(II)
                    n=int( len(II)/2 )
                    
                    if n>=nmim:
                        print('final',i,j,n)
                        final_edges=final_edges.append(
                            {'s':i,'t':j,'n':n},ignore_index=True).astype(int)
                    if i==625 and j==307561:
                        sys.exit('iii {} {}'.format(i,j))
        if True:
            index_to_drop=[]
            pass

        di=list(pjr.index)+index_to_drop
        pj=pj.drop(pj.index[I])

n_i 0
625 307561
final 625 307561 26


SystemExit: iii 625 307561

In [558]:
pji.index

Int64Index([  0,   1,  10,  11,  18,  19,  44,  45,  50,  51,  68,  69,  72,
             73,  75,  76,  77,  78,  79,  80,  92,  93, 185, 186, 187, 188,
            209, 210, 216, 217, 222, 223, 229, 230, 274, 275, 277, 278, 285,
            286, 293, 294, 296, 297, 311, 312, 313, 314, 321, 322, 769, 770],
           dtype='int64')

In [560]:
pji.shape,II

((52, 4),
 array([  0,   1,  10,  11,  18,  19,  44,  45,  50,  51,  68,  69,  72,
         73,  75,  76,  77,  78,  79,  80,  92,  93, 185, 186, 187, 188,
        209, 210, 216, 217, 222, 223, 229, 230, 274, 275, 277, 278, 285,
        286, 293, 294, 296, 297, 311, 312, 313, 314, 321, 322, 769, 770]))

In [530]:
import sys

In [470]:
pj=pj.sort_values(['references','recid']).reset_index(drop=True) 

In [471]:
pj[:10]

,recid,references
0,76215,1
1,81651,1
2,128250,1
3,139190,1
4,151315,1
5,275304,1
6,311629,1
7,366639,1
8,452267,1
9,524107,1


In [473]:
get_edges(pj,76215,81651)

,recid,references,nm,np
0,76215,1,0,1
1,81651,1,1,0


In [413]:
pj=pj.drop(pj.index[I])#.sort_values(['references','recid']).reset_index(drop=True)

In [414]:
pj.shape

(21733765, 2)

In [409]:
df

""


In [402]:
df[ df.n>1 ]

AttributeError: 'DataFrame' object has no attribute 'n'

In [234]:
dm=pji.references.values[:-1]-pji.references.values[1:]
pji['nm']=np.concatenate((dm,[1]))
pji['np']=np.concatenate(([1],dm))
pji=pji[np.logical_or(pji.nm==0,pji.np==0)]

I=pji.index
n=pji.shape[0]/2
if n>=nmim:
    df=df.append({'s':i,'t':j,'n':n},ignore_index=True)

In [233]:
df

""


In [222]:
ppj

,recid,references,nm,np
0,284861,113,0,1
1,487322,113,1,0


In [7]:
#pj=pj[:4000074]#.reset_index(drop=True)

In [10]:
iini=0
iend=1000000
for i in range(1000):
    if i==0:
        ref_old=pj.loc[iend,'references']

    if pj.loc[iend,'references']!=ref_old:
        break
    iend=iend+i

pj=pj[iini:iend-1].reset_index(drop=True)
iini=iend-1    
iend=iend+1000000

In [23]:
ppj=pj_full[np.logical_or(pj_full.recid==76215,pj_full.recid==81651)]
ppj.shape

(2, 2)

In [25]:
ppj.transpose()

,0,1
recid,76215,81651
references,1,1


In [36]:
pj[200000:200000+10]

,recid,references
200000,866689,4510
200001,867097,4510
200002,867121,4510
200003,867124,4510
200004,867129,4510
200005,867130,4510
200006,867131,4510
200007,867137,4510
200008,867150,4510
200009,867156,4510


In [489]:
#: # 
i,j=625, 307561
df=pj_full.copy()
def get_edges(df,i,j):
    dfi=df[np.logical_or(df.recid==i,df.recid==j)]
    dm=dfi.references.values[:-1]-dfi.references.values[1:]
    dfi['nm']=np.concatenate((dm,[1]))
    dfi['np']=np.concatenate(([1],dm))
    return dfi[np.logical_or(dfi.nm==0,dfi.np==0)]

In [492]:
pj

,recid,references
10691,307561,719
1102215,307561,37141
1160959,307561,40076
1161832,307561,40077
1163126,307561,40078
1166984,307561,40208
1168091,307561,40216
1171809,625,40433
1172290,307561,40433
1174364,625,40440


In [475]:
ppj=pj_full[np.logical_or(pj_full.recid==1403659,pj_full.recid==1244602)]
ppj=ppj.sort_values(['references','recid'])
ppj.shape

(74, 2)

In [372]:
ppj

,recid,references
18,1244602,32
19,1403659,32
504602,1403659,12474
771548,1403659,22740
3605598,1244602,165994
4370720,1403659,206433
4976456,1244602,245541
6408982,1244602,339331
6952046,1403659,373454
7078482,1244602,378794


In [ ]:
625, 307561

In [44]:
ppj=pj_full[np.logical_or(pj_full.recid==625,pj_full.recid==307561)]
ppj=ppj.sort_values(['references','recid'])
ppj.shape

(775, 2)

In [59]:
mc=ppj.references[1:].values-ppj.references[:-1].values

In [61]:
mc[mc==0].shape

(26,)

In [62]:
diff=ppj.references[1:].values

In [68]:
ppj['nm']=np.concatenate((mc,[1]))
ppj['np']=np.concatenate(([1],mc))

In [71]:
ppj[np.logical_or(ppj.nm==0,ppj.np==0)]

,recid,references,n,nm,np
1577,625,113,0,0,1
2696,307561,113,606,606,0
86805,625,2451,0,0,28
87149,307561,2451,26,26,0
179915,625,4327,0,0,421
181189,307561,4327,1,1,0
342916,625,9113,0,0,56
343074,307561,9113,9,9,0
360567,625,9154,0,0,5
360794,307561,9154,15,15,0


##  Main code to find the edges

In [9]:
start_time=time.time()

DEBUG=True
if DEBUG: 
    #===log===
    f=open('kk.log','w')
    f.write('')
    f.close()
    #==DEBUG===========

id_col='recid'; refs_col='references'
# Half of the work is order by refs_col, and id_col
pj=pj.sort_values([refs_col,id_col]).reset_index(drop=True)
ed={}
t=()
new_ref=0
i=0
iprint=1000

while i<=pj.index[-1]:
    if not i%iprint:
        print(i,end='\r')
        if DEBUG:
            #===log===
            f=open('kk.log','a')
            f.write('{}\n'.format(i))
            f.close()
            #=======
    e1=pj.loc[i,id_col]
    #check next entry and beyond:
    for j in range(i+1,pj.index[-1]+1):
        #print('i,j',i,j) #,end='\r')
        e2=pj.loc[j,id_col]
        #DEBUG
        #if (e1,e2)==(190836,1644265):
        #    print('i,j,check',i,j)
        edges=False
        if pj.loc[i,refs_col]==pj.loc[j,refs_col]:
            edges=True
        #else:
            #print('newi',i,j)
            
        if edges:
            if e1<e2:
                t=(e1,e2)
                if not ed.get(t):
                    ed[t]=1
                    #DEBUG
                    #ed[t]=str( (i,j) )
                else: #increase the number of common refs_col
                    ed[t]=ed[t]+1
                    #DEBUG: 
                    #ed[t]=ed[t]+str(  (i,j)  )
        else:
            # Go to next i when nor additional edges are found
            break
    #search after j. 
    i=i+1 # Be sure to increase i to avoid infinite loop
    #if i>=25:
    #    break
    continue

print(time.time()-start_time)    

15287.510861873627


In [10]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

655851

In [ ]:
1+1

In [14]:
list(ed.keys())[:3]

[(802724, 1190685), (664030, 809979), (1186162, 1208546)]

Continue in [run2](./run2/pajek.ipynb)

In [47]:
N=lambda x:  x*1052/100000/60
npt=1000000
print( N(npt),'m') 
print( N(npt)/60,'h') 

175.33333333333334 m
2.9222222222222225 h


In [48]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

3018

In [49]:
edf=pd.DataFrame({'tmp':le})
edf['edges'] =edf.tmp.map(lambda x: list(x.keys())[0]  )
edf['len_ed']=edf.tmp.map(lambda x: list(x.values())[0]  )
edf=edf.drop('tmp',axis='columns')
edf[:3]

,edges,len_ed
0,"(4380, 666911)",5
1,"(15239, 1387684)",5
2,"(98947, 129333)",5


In [50]:
#edf.to_json('example1E5.json')

In [99]:
pd.DataFrame([{'ttbar':
{'tau pT' : r'$3.29 pm 0.0056$',
'b-jet' : r'$49.8 pm 0.087$',
'lep-veto' : r'$71.8 pm 0.11$',
'MET' : r'$12.4 pm 0.096$',
'Delta phi' : r'$24.8 pm 0.36$'}},
              {'single top':
{'tau pT' : r'$1.13 pm 0.0035$',
'b-jet' : r'$40.4 pm 0.15$',
'lep-veto' : r'$90.5 pm 0.14$',
'MET' : r'$2.55 pm 0.082$',
'Delta phi' : r'$31.4 pm 1.5$'}}
             ])

,single top,ttbar
0,NaN,"{'MET': '$12.4 pm 0.096$', 'tau pT': '$3.29 pm..."
1,"{'MET': '$2.55 pm 0.082$', 'tau pT': '$1.13 pm...",NaN


In [106]:
pd.DataFrame([{'N':{'ttbar':
[{'tau pT' : r'$3.29 pm 0.0056$',
'b-jet' : r'$49.8 pm 0.087$',
'lep-veto' : r'$71.8 pm 0.11$',
'MET' : r'$12.4 pm 0.096$',
'Delta phi' : r'$24.8 pm 0.36$'}]}}])

,N
0,"{'ttbar': [{'MET': '$12.4 pm 0.096$', 'tau pT'..."


In [188]:
l=[{'ttbar':
{'tau pT' : r'$3.29 \pm 0.0056$',
'b-jet' : r'$49.8 \pm 0.087$',
'lep-veto' : r'$71.8 \pm 0.11$',
'MET' : r'$12.4 \pm 0.096$',
'Delta phi' : r'$24.8 \pm 0.36$'}},
{'single top':
{'tau pT' : r'$1.13 \pm 0.0035$',
'b-jet' : r'$40.4 \pm 0.15$',
'lep-veto' : r'$90.5 \pm 0.14$',
'MET' : r'$2.55 \pm 0.082$',
'Delta phi' : r'$31.4 \pm 1.5$'}},
{'w+jets':
{'tau pT' : r'$2.97 \pm 0.0094$',
'b-jet' : r'$8.4 \pm 0.09$',
'lep-veto' : r'$94.1 \pm 0.26$',
'MET' : r'$9.65 \pm 0.34$',
'Delta phi' : r'$23 \pm 1.6$'}},
{'z+jets':
{'tau pT' : r'$2.87 \pm 0.014$',
'b-jet' : r'$14.1 \pm 0.18$',
'lep-veto' : r'$97.4 \pm 0.21$',
'MET' : r'$6.45 \pm 0.34$',
'Delta phi' : r'$32.4 \pm 2.5$'}},
{'ww':
{'tau pT' : r'$0.575 \pm 0.0042$',
'b-jet' : r'$7.66 \pm 0.19$',
'lep-veto' : r'$92.5 \pm 0.7$',
'MET' : r'$6.63 \pm 0.68$',
'Delta phi' : r'$21.6 \pm 4.4$'}},
{'wz':
{'tau pT' : r'$0.638 \pm 0.0071$',
'b-jet' : r'$11.9 \pm 0.36$',
'lep-veto' : r'$93.2 \pm 0.82$',
'MET' : r'$6.7 \pm 0.84$',
'Delta phi' : r'$39 \pm 6.3$'}},
{'zz':
{'tau pT' : r'$0.673 \pm 0.012$',
'b-jet' : r'$17.8 \pm 0.67$',
'lep-veto' : r'$92.7 \pm 1.1$',
'MET' : r'$6 \pm 1$',
'Delta phi' : r'$65.6 \pm 8.4$'}},
{'signal 200':
{'tau pT' : r'$8.34 \pm 0.081$',
'b-jet' : r'$18.1 \pm 0.39$',
'lep-veto' : r'$99.6 \pm 0.15$',
'MET' : r'$5.97 \pm 0.57$',
'Delta phi' : r'$25.7 \pm 4.3$'}},
{'signal 250':
{'tau pT' : r'$13.9 \pm 0.15$',
'b-jet' : r'$17.2 \pm 0.44$',
'lep-veto' : r'$99.9 \pm 0.078$',
'MET' : r'$15.1 \pm 1$',
'Delta phi' : r'$49 \pm 3.6$'}},
{'signal 300':
{'tau pT' : r'$18.3 \pm 0.24$',
'b-jet' : r'$17.4 \pm 0.56$',
'lep-veto' : r'$99.9 \pm 0.13$',
'MET' : r'$28.6 \pm 1.6$',
'Delta phi' : r'$69.2 \pm 3.1$'}},
{'signal 350':
{'tau pT' : r'$22.2 \pm 0.27$',
'b-jet' : r'$17.1 \pm 0.52$',
'lep-veto' : r'$99.7 \pm 0.19$',
'MET' : r'$37.6 \pm 1.6$',
'Delta phi' : r'$67.8 \pm 2.5$'}},
{'signal 500':
{'tau pT' : r'$27.5 \pm 0.31$',
'b-jet' : r'$19.6 \pm 0.52$',
'lep-veto' : r'$99.9 \pm 0.089$',
'MET' : r'$61.5 \pm 1.4$',
'Delta phi' : r'$77.5 \pm 1.6$'}},
{'signal 750':
{'tau pT' : r'$31.5 \pm 0.34$',
'b-jet' : r'$21.7 \pm 0.54$',
'lep-veto' : r'$99.7 \pm 0.16$',
'MET' : r'$79.1 \pm 1.2$',
'Delta phi' : r'$82.4 \pm 1.2$'}},
{'signal 1000':
{'tau pT' : r'$32.8 \pm 0.37$',
'b-jet' : r'$21.6 \pm 0.56$',
'lep-veto' : r'$99.7 \pm 0.17$',
'MET' : r'$87.2 \pm 0.98$',
'Delta phi' : r'$83.4 \pm 1.2$'}}]

In [189]:
inds=[]
for i in range(len(l)):
    if i==0:
        cols=list( l[0]['ttbar'].keys() )
    ind_i=list( l[i].keys() )[0]
    inds.append( ind_i )
    
df=pd.DataFrame(index=inds,columns=cols).fillna('')
df

,MET,tau pT,lep-veto,b-jet,Delta phi
ttbar,,,,,
single top,,,,,
w+jets,,,,,
z+jets,,,,,
ww,,,,,
wz,,,,,
zz,,,,,
signal 200,,,,,
signal 250,,,,,
signal 300,,,,,


In [190]:
i=0
for index in inds:
    for columns in cols:
        df.loc[index,columns]=l[i][index][columns]
    i=i+1

In [191]:
print(df.to_latex())

\begin{tabular}{llllll}
\toprule
{} &               MET &              tau pT &          lep-veto &             b-jet &        Delta phi \\
\midrule
ttbar       &  \$12.4 \textbackslashpm 0.096\$ &   \$3.29 \textbackslashpm 0.0056\$ &   \$71.8 \textbackslashpm 0.11\$ &  \$49.8 \textbackslashpm 0.087\$ &  \$24.8 \textbackslashpm 0.36\$ \\
single top  &  \$2.55 \textbackslashpm 0.082\$ &   \$1.13 \textbackslashpm 0.0035\$ &   \$90.5 \textbackslashpm 0.14\$ &   \$40.4 \textbackslashpm 0.15\$ &   \$31.4 \textbackslashpm 1.5\$ \\
w+jets      &   \$9.65 \textbackslashpm 0.34\$ &   \$2.97 \textbackslashpm 0.0094\$ &   \$94.1 \textbackslashpm 0.26\$ &    \$8.4 \textbackslashpm 0.09\$ &     \$23 \textbackslashpm 1.6\$ \\
z+jets      &   \$6.45 \textbackslashpm 0.34\$ &    \$2.87 \textbackslashpm 0.014\$ &   \$97.4 \textbackslashpm 0.21\$ &   \$14.1 \textbackslashpm 0.18\$ &   \$32.4 \textbackslashpm 2.5\$ \\
ww          &   \$6.63 \textbackslashpm 0.68\$ &  \$0.575 \textbackslashpm 0.0042\$ &  

In [192]:
df.to_latex?

In [171]:
df=pd.DataFrame(index=ind,columns=cols).fillna('')
df

,MET,tau pT,lep-veto,b-jet,Delta phi
ttbar,,,,,
single top,,,,,
w+jets,,,,,
z+jets,,,,,
ww,,,,,
wz,,,,,
zz,,,,,
signal 200,,,,,
signal 250,,,,,
signal 300,,,,,


In [172]:
df.loc['ww','MET']

''

In [136]:
df.N[0].get('ttbar')

{'Delta phi': '$24.8 pm 0.36$',
 'MET': '$12.4 pm 0.096$',
 'b-jet': '$49.8 pm 0.087$',
 'lep-veto': '$71.8 pm 0.11$',
 'tau pT': '$3.29 pm 0.0056$'}

In [140]:
df

,N
0,"{'ttbar': {'MET': '$12.4 pm 0.096$', 'tau pT':..."
1,"{'single top': {'MET': '$2.55 pm 0.082$', 'tau..."
2,"{'w+jets': {'MET': '$9.65 pm 0.34$', 'tau pT':..."
3,"{'z+jets': {'MET': '$6.45 pm 0.34$', 'tau pT':..."
4,"{'ww': {'MET': '$6.63 pm 0.68$', 'tau pT': '$0..."
5,"{'wz': {'MET': '$6.7 pm 0.84$', 'tau pT': '$0...."
6,"{'zz': {'MET': '$6 pm 1$', 'tau pT': '$0.673 p..."
7,"{'signal 200': {'MET': '$5.97 pm 0.57$', 'tau ..."
8,"{'signal 250': {'MET': '$15.1 pm 1$', 'tau pT'..."
9,"{'signal 300': {'MET': '$28.6 pm 1.6$', 'tau p..."


In [133]:
df.N[[0]].get(0)

{'ttbar': {'Delta phi': '$24.8 pm 0.36$',
  'MET': '$12.4 pm 0.096$',
  'b-jet': '$49.8 pm 0.087$',
  'lep-veto': '$71.8 pm 0.11$',
  'tau pT': '$3.29 pm 0.0056$'}}

In [120]:
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

unpack(df, 'N', 0)

,signal 1000,signal 200,signal 250,signal 300,signal 350,signal 500,signal 750,single top,ttbar,w+jets,ww,wz,z+jets,zz
0,0,0,0,0,0,0,0,0,"{'MET': '$12.4 pm 0.096$', 'tau pT': '$3.29 pm...",0,0,0,0,0
1,0,0,0,0,0,0,0,"{'MET': '$2.55 pm 0.082$', 'tau pT': '$1.13 pm...",0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,"{'MET': '$9.65 pm 0.34$', 'tau pT': '$2.97 pm ...",0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,"{'MET': '$6.45 pm 0.34$', 'tau pT': '$2.87 pm ...",0
4,0,0,0,0,0,0,0,0,0,0,"{'MET': '$6.63 pm 0.68$', 'tau pT': '$0.575 pm...",0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,"{'MET': '$6.7 pm 0.84$', 'tau pT': '$0.638 pm ...",0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'MET': '$6 pm 1$', 'tau pT': '$0.673 pm 0.012..."
7,0,"{'MET': '$5.97 pm 0.57$', 'tau pT': '$8.34 pm ...",0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,"{'MET': '$15.1 pm 1$', 'tau pT': '$13.9 pm 0.1...",0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,"{'MET': '$28.6 pm 1.6$', 'tau pT': '$18.3 pm 0...",0,0,0,0,0,0,0,0,0,0


Go to the previous code and run there. 

## Backup

IMPORTANT COMMITS: 
* 44fa444ba3cc1f9a052bf80051f44a0d11c5838a
  * Here the counting of the slow and fast to grouping mathc
